<a href="https://colab.research.google.com/github/kekele-star/fake-news-pred/blob/main/fake_news_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

About the Dataset
  1. id: unique id for a news article
  2. title: the title of a news article
  3. author: author of the news article
  4. text: the text of the article; could be incomplete
  5. label: a label that marks whether the news article is real or fake:

          1: Fake news
          0: real news

## Importing Dependencies

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


import re

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [1]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
# print the stopwords in English
print(stopwords.words("english"))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

## Load Dataset

## Data Pre-processing


In [9]:
news_dataset = pd.read_csv('/content/train.csv')
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [11]:
news_dataset.shape

(20800, 5)

In [12]:
news_dataset.info

<bound method DataFrame.info of           id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiumnews.com   
3                                Jessica Purkiss   
4                                 Howard Portnoy   
...                                          ...   
20795                              Jerome Hudson   
20796                           Benjamin Hoffman   
20797  Michael J. de la Merced and Rachel Abrams   
20798                                Alex Ansary   
20799                              David Swanson   

                                                    text  label  
0      House Dem Aide: We Didn’t Even See Comey’s Let...      1  
1      Ever get the feeling your life circles the rou...      0  
2      Why the Truth Might Get You Fired October 29, ...      1  
3      Videos 15 Civilians Killed In Single US Airstr...      1  
4      Print \nAn Iranian woman has been sentenced to...      1  
...                                                  ...    ...  
20795  Rapper T. I. unloaded on black celebrities who...      0  
20796  When the Green Bay Packers lost to the Washing...      0  
20797  The Macy’s of today grew from the union of sev...      0  
20798  NATO, Russia To Hold Parallel Exercises In Bal...      1  
20799    David Swanson is an author, activist, journa...      1  

[20800 rows x 5 columns]>

In [13]:
# count number of missing values
news_dataset.isnull().sum()

,0
id,0
title,558
author,1957
text,39
label,0


In [14]:
# replace null values with empty string
news_dataset = news_dataset.fillna('')

In [15]:
news_dataset.isnull().sum()

,0
id,0
title,0
author,0
text,0
label,0


In [16]:
# merge the author name and news title
news_dataset['content'] = news_dataset['author']+' '+news_dataset['title']

In [17]:
# separate the data & label
X = news_dataset.drop(columns='label', axis=1) # axis = 1 to remove column, if to remove row, axis = 0
Y = news_dataset['label'] # assign labels to "Y"

In [18]:
print(X)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

In [19]:
print(Y)

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 20800, dtype: int64


## Steaming

Steaming: is the process of reducing a word to its Root word

example: actor, actress, acting --> act

In [20]:
port_steam = PorterStemmer()

The `stemming` function below processes a text string by first removing any non-alphabetic characters and converting the text to lowercase. It then splits the text into individual words and applies stemming, which reduces each word to its root form, using the PorterStemmer algorithm.

Common stopwords, like "and" or "the," are filtered out during this process. Finally, the stemmed words are rejoined into a single string.


In [22]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA)z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_steam.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content



In [23]:
news_dataset['content'] = news_dataset['content'].apply(stemming)

In [25]:
print(news_dataset['content'])

0        arrel ucu ous em aid e idn ven ee omey etter n...
1           aniel lynn illari linton ig oman ampu reitbart
2                  onsortiumnew com hy ruth ight et ou ire
3        essica urkiss ivilian ill n ingl airstrik ave ...
4        oward ortnoy ranian woman jail fiction unpubli...
                               ...                        
20795       erom udson apper rump oster hild hite upremaci
20796    enjamin offman layoff chedul atchup dd ew ork ime
20797    ichael de la erc achel abram aci aid eceiv ake...
20798         alex ansari ussia old arallel xercis n alkan
20799                             avid wanson hat eep aliv
Name: content, Length: 20800, dtype: object


In [34]:
# separating the data and label
X = news_dataset['content'].values
Y = news_dataset['label'].values

In [35]:
print(X)

['arrel ucu ous em aid e idn ven ee omey etter ntil ason haffetz weet'
 'aniel lynn illari linton ig oman ampu reitbart'
 'onsortiumnew com hy ruth ight et ou ire' ...
 'ichael de la erc achel abram aci aid eceiv akeov approach udson ay ew ork ime'
 'alex ansari ussia old arallel xercis n alkan' 'avid wanson hat eep aliv']


In [36]:
print(Y)

[1 0 1 ... 0 1 1]


In [37]:
Y.shape

(20800,)

## Transorm Textual data to Numeric data

In [38]:
# converting the textual data to numeric data for the computer to understand
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [39]:
print(X)

  (0, 16187)	0.2751136106666613
  (0, 15967)	0.25721451035053505
  (0, 15088)	0.330956879736425
  (0, 11899)	0.2018697211989392
  (0, 10934)	0.22863312602540892
  (0, 10295)	0.3022365599314355
  (0, 7442)	0.27837634506463166
  (0, 6559)	0.34682278796528043
  (0, 5901)	0.25344000620589097
  (0, 5007)	0.24918253458902978
  (0, 4550)	0.21151494630800288
  (0, 2291)	0.2189585459679329
  (0, 2082)	0.31732758849472026
  (0, 592)	0.21625884494417633
  (1, 13074)	0.1989230529846486
  (1, 10879)	0.390175749815279
  (1, 9264)	0.4577275079512919
  (1, 9019)	0.24717429791998466
  (1, 7696)	0.2459555256395249
  (1, 7523)	0.35008762640043456
  (1, 1401)	0.3375557254849401
  (1, 1204)	0.4904931840513633
  (2, 13820)	0.38505168207348944
  (2, 11795)	0.32106313955963073
  (2, 11181)	0.45862337972873657
  :	:
  (20797, 7772)	0.08682805757065797
  (20797, 7334)	0.18580542633764677
  (20797, 6002)	0.08189247554098847
  (20797, 5450)	0.31422031573094805
  (20797, 4389)	0.2940125288564591
  (20797, 3857)	0.

## Split the dataset to training and testing set

In [40]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify=Y, random_state=2)

## Train the Model --> Logistic Regression

In [41]:
model = LogisticRegression()

In [42]:
model.fit(X_train, Y_train)

LogisticRegression()

## Evaluate Model

In [43]:
# accuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)